# 3 Classificação e previsão 
## 3.1) Quão maior os gastos e investimentos com a educação, menor será a taxa de analfabetismo?

In [99]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

## 3.1.1) Carga e transformação da massa bruta dos dados com pelo menos 50% dos valores preenchidos.

In [113]:
massa_bruta_pais_por_indicadores = pd.read_csv('../data/massa_bruta_pais_por_indicadores.csv')

anos = ['1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', 
        '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']

indicadores = ['UIS.LP.AG15T24', 'UIS.ILLPOP.AG25T64', 'UIS.LP.AG65',
               'UIS.XSPENDP.FDPUB.FNS', 'UIS.XSPENDP.FDPUB.FNCAP', 'UIS.XSPENDP.FDPUB.FNNONS', 
               'UIS.XGDP.FSGOV.FDINSTADM.FFD', 'SE.XPD.TOTL.GD.ZS']

paises = massa_bruta_pais_por_indicadores['Country Name'].unique()

df_massa_bruta_por_indicador = massa_bruta_pais_por_indicadores.loc[massa_bruta_pais_por_indicadores['Indicator Code'].isin(indicadores)]

colunas_transformacao = ['Pais', 'Indicador', '1998-1999', '1999-2000', '2000-2001', '2001-2002', 
                        '2002-2003', '2003-2004', '2004-2005', '2005-2006', '2006-2007', '2007-2008', 
                        '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013']

df_resultado_transformacao = pd.DataFrame(columns=colunas_transformacao)


for pais in paises:
    df_pais = df_massa_bruta_por_indicador.loc[df_massa_bruta_por_indicador['Country Name'] == pais]

    df_1998_a_2013 = df_pais.iloc[:, 2:18]
    df_1998_a_2013.index = range(df_1998_a_2013.shape[0])
    media_valores_invalidos = df_1998_a_2013.isna().mean(axis=1).mean()
#     df_1998_a_2013 = df_1998_a_2013.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).fillna(0)
    
    if media_valores_invalidos < 0.5:
        
        df_1998_a_2013 = df_1998_a_2013.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).fillna(0)    
    
        indicador = df_pais['Indicator Code']
        indicador.index = range(indicador.shape[0])

        for index, value in indicador.items():
            valor_1998_1999 = 1 if df_1998_a_2013['1999'].iloc[0] - df_1998_a_2013['1998'].iloc[0] > 0 else 0
            valor_1999_2000 = 1 if df_1998_a_2013['2000'].iloc[0] - df_1998_a_2013['1999'].iloc[0] > 0 else 0
            valor_2000_2001 = 1 if df_1998_a_2013['2001'].iloc[0] - df_1998_a_2013['2000'].iloc[0] > 0 else 0
            valor_2001_2002 = 1 if df_1998_a_2013['2002'].iloc[0] - df_1998_a_2013['2001'].iloc[0] > 0 else 0
            valor_2002_2003 = 1 if df_1998_a_2013['2003'].iloc[0] - df_1998_a_2013['2002'].iloc[0] > 0 else 0
            valor_2003_2004 = 1 if df_1998_a_2013['2004'].iloc[0] - df_1998_a_2013['2003'].iloc[0] > 0 else 0
            valor_2004_2005 = 1 if df_1998_a_2013['2005'].iloc[0] - df_1998_a_2013['2004'].iloc[0] > 0 else 0
            valor_2005_2006 = 1 if df_1998_a_2013['2006'].iloc[0] - df_1998_a_2013['2005'].iloc[0] > 0 else 0
            valor_2006_2007 = 1 if df_1998_a_2013['2007'].iloc[0] - df_1998_a_2013['2006'].iloc[0] > 0 else 0
            valor_2007_2008 = 1 if df_1998_a_2013['2008'].iloc[0] - df_1998_a_2013['2007'].iloc[0] > 0 else 0
            valor_2008_2009 = 1 if df_1998_a_2013['2009'].iloc[0] - df_1998_a_2013['2008'].iloc[0] > 0 else 0
            valor_2009_2010 = 1 if df_1998_a_2013['2010'].iloc[0] - df_1998_a_2013['2009'].iloc[0] > 0 else 0
            valor_2010_2011 = 1 if df_1998_a_2013['2011'].iloc[0] - df_1998_a_2013['2010'].iloc[0] > 0 else 0
            valor_2011_2012 = 1 if df_1998_a_2013['2012'].iloc[0] - df_1998_a_2013['2011'].iloc[0] > 0 else 0
            valor_2012_2013 = 1 if df_1998_a_2013['2013'].iloc[0] - df_1998_a_2013['2012'].iloc[0] > 0 else 0

            valores = [pais, value, valor_1998_1999, valor_1999_2000, valor_2000_2001, valor_2001_2002, 
                       valor_2002_2003, valor_2003_2004, valor_2004_2005, valor_2005_2006, valor_2006_2007, 
                       valor_2007_2008, valor_2008_2009, valor_2009_2010, valor_2010_2011, valor_2011_2012, 
                       valor_2012_2013]

            df_auxiliar = pd.DataFrame([valores], columns=colunas_transformacao)
            df_auxiliar.index = range(df_auxiliar.shape[0])
            df_resultado_transformacao = pd.concat([df_auxiliar, df_resultado_transformacao])
    
df_resultado_transformacao = df_resultado_transformacao.sort_values(['Pais'], ascending=[True])
df_resultado_transformacao.index = range(df_resultado_transformacao.shape[0])

### 3.1.1.1) Transformação e seleção para o indicador de analfabetismo entre 15 a 24 anos.

In [118]:
indicadores_15_a_24 = ['UIS.LP.AG15T24', 'UIS.XSPENDP.FDPUB.FNS', 'UIS.XSPENDP.FDPUB.FNCAP', 
                       'UIS.XSPENDP.FDPUB.FNNONS', 'UIS.XGDP.FSGOV.FDINSTADM.FFD', 'SE.XPD.TOTL.GD.ZS']

df_analfabetismo_15_a_24 = pd.DataFrame()

for indicador in indicadores_15_a_24:
    df_indicador = df_resultado_transformacao.loc[df_resultado_transformacao['Indicador'] == indicador]
    df_periodos_indicador = df_indicador.drop(['Pais', 'Indicador'], axis=1)    
    df_periodos_indicador = df_periodos_indicador.stack()#.reset_index()
    df_periodos_indicador.index = range(df_periodos_indicador.shape[0])
    df_analfabetismo_15_a_24[indicador] = df_periodos_indicador.tolist()

nomes_colunas = {'UIS.LP.AG15T24': 'Analfabetismo_15_24', 
                 'UIS.ILLPOP.AG25T64': 'Analfabetismo_25_64', 
                 'UIS.LP.AG65': 'Analfabetismo_65_mais', 
                 'UIS.XSPENDP.FDPUB.FNS': 'Gastos_Pessoas',
                 'UIS.XSPENDP.FDPUB.FNCAP': 'Gastos_Totais', 
                 'UIS.XSPENDP.FDPUB.FNNONS': 'Gastos_Logística',
                 'UIS.XGDP.FSGOV.FDINSTADM.FFD': 'Gastos_Infraestrutura', 
                 'SE.XPD.TOTL.GD.ZS': 'Gastos_Governo'}
    
df_analfabetismo_15_a_24 = df_analfabetismo_15_a_24.rename(columns = nomes_colunas)

### 3.1.1.2) Transformação e seleção para o indicador de analfabetismo entre 25 a 64 anos.

In [115]:
indicadores_25_a_64 = ['UIS.ILLPOP.AG25T64', 'UIS.XSPENDP.FDPUB.FNS', 'UIS.XSPENDP.FDPUB.FNCAP', 
                       'UIS.XSPENDP.FDPUB.FNNONS', 'UIS.XGDP.FSGOV.FDINSTADM.FFD', 'SE.XPD.TOTL.GD.ZS']

df_analfabetismo_25_a_64 = pd.DataFrame()

for indicador in indicadores_25_a_64:
    df_indicador = df_resultado_transformacao.loc[df_resultado_transformacao['Indicador'] == indicador]
    df_periodos_indicador = df_indicador.drop(['Pais', 'Indicador'], axis=1)    
    df_periodos_indicador = df_periodos_indicador.stack()#.reset_index()
    df_periodos_indicador.index = range(df_periodos_indicador.shape[0])
    df_analfabetismo_25_a_64[indicador] = df_periodos_indicador.tolist()

df_analfabetismo_25_a_64 = df_analfabetismo_25_a_64.rename(columns = nomes_colunas)
df_analfabetismo_25_a_64

,Analfabetismo_25_64,Gastos_Pessoas,Gastos_Totais,Gastos_Logística,Gastos_Infraestrutura,Gastos_Governo
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,1,1,1,1,1,1
4,0,0,0,0,0,0
...,...,...,...,...,...,...
505,0,0,0,0,0,0
506,0,0,0,0,0,0
507,1,1,1,1,1,1
508,0,0,0,0,0,0


### 3.1.1.3) Transformação e seleção para o indicador de analfabetismo para 65 anos ou mais.

In [112]:
indicadores_65 = ['UIS.LP.AG65', 'UIS.XSPENDP.FDPUB.FNS', 'UIS.XSPENDP.FDPUB.FNCAP', 
                  'UIS.XSPENDP.FDPUB.FNNONS', 'UIS.XGDP.FSGOV.FDINSTADM.FFD', 'SE.XPD.TOTL.GD.ZS']

df_analfabetismo_65 = pd.DataFrame()

for indicador in indicadores_65:
    df_indicador = df_resultado_transformacao.loc[df_resultado_transformacao['Indicador'] == indicador]
    df_periodos_indicador = df_indicador.drop(['Pais', 'Indicador'], axis=1)    
    df_periodos_indicador = df_periodos_indicador.stack()#.reset_index()
    df_periodos_indicador.index = range(df_periodos_indicador.shape[0])
    df_analfabetismo_65[indicador] = df_periodos_indicador.tolist()

df_analfabetismo_65 = df_analfabetismo_65.rename(columns = nomes_colunas)

## 3.1.1.1) Validação da eficiência da previsão

In [9]:
#6) Aplicar algoritmo de classificação e previsão


#Fatos:
#1) As linhas do dataframe estão agrupadas por países.
#2) Cada país possui uma lista de indicadores.
#3) A lista de indicadores é dividida em dois grupos:
#   Grupo 1 = indicadores de analfabetismo (influenciados): UIS.LP.AG15T24, UIS.ILLPOP.AG25T64, UIS.LP.AG65
#   Grupo 2 = indicadores econômicos (influenciadores): UIS.XSPENDP.FDPUB.FNS, UIS.XSPENDP.FDPUB.FNCAP, UIS.XSPENDP.FDPUB.FNNONS, UIS.XGDP.FSGOV.FDINSTADM.FFD, SE.XPD.TOTL.GD.ZS
#4) Além disso, existem as colunas (com e 1) da diferença entre os anos de 1998 a 2013.

#O quê fazer?
#1) Transformar o dataset em:
#   Linhas = período de anos (1998-1999, 1999-2000, ..., 2012-2013)
#   Colunas = um indicador por coluna: UIS.LP.AG15T24, UIS.ILLPOP.AG25T64, UIS.LP.AG65, UIS.XSPENDP.FDPUB.FNS, UIS.XSPENDP.FDPUB.FNCAP, UIS.XSPENDP.FDPUB.FNNONS, UIS.XGDP.FSGOV.FDINSTADM.FFD, SE.XPD.TOTL.GD.ZS


df_periodo_datas = df_resultado_transformacao.iloc[:, 2:18]


SEED = 20

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y,
                                                         random_state = SEED, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

,Pais,Indicador,1998-1999,1999-2000,2000-2001,2001-2002,2002-2003,2003-2004,2004-2005,2005-2006,2006-2007,2007-2008,2008-2009,2009-2010,2010-2011,2011-2012,2012-2013
0,Afghanistan,UIS.XSPENDP.FDPUB.FNS,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,Afghanistan,UIS.LP.AG15T24,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,Afghanistan,UIS.ILLPOP.AG25T64,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,Afghanistan,SE.XPD.TOTL.GD.ZS,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,Afghanistan,UIS.XSPENDP.FDPUB.FNCAP,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,Zimbabwe,SE.XPD.TOTL.GB.ZS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1949,Zimbabwe,UIS.XGDP.FSGOV.FDINSTADM.FFD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1950,Zimbabwe,SE.XPD.TOTL.GD.ZS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1951,Zimbabwe,UIS.ILLPOP.AG25T64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 3.1.2) Carga e transformação da massa bruta dos dados com pelo menos 50% dos valores preenchidos.

Isso reduzirá a quantidade de elementos para 351. Na solução 3.1.1, sem a restrição de 50% do preenchimento, foram utilizados 1953 elementos.

In [48]:
massa_bruta_pais_por_indicadores = pd.read_csv('../data/massa_bruta_pais_por_indicadores.csv')

paises = massa_bruta_pais_por_indicadores['Country Name'].unique()

df_massa_bruta_por_indicador = massa_bruta_pais_por_indicadores.loc[massa_bruta_pais_por_indicadores['Indicator Code'].isin(indicadores)]

df_classificado = pd.DataFrame(columns=colunas_transformacao)

for pais in paises:
    df_pais = df_massa_bruta_por_indicador.loc[df_massa_bruta_por_indicador['Country Name'] == pais]

    df_1998_a_2013 = df_pais.iloc[:, 2:18]
    df_1998_a_2013.index = range(df_1998_a_2013.shape[0])
    media_valores_invalidos = df_1998_a_2013.isna().mean(axis=1).mean()
    
    if media_valores_invalidos < 0.5:
        
        df_1998_a_2013 = df_1998_a_2013.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).fillna(0)
        
        for index, value in indicador.items():            
            valor_1998_1999 = 1 if df_1998_a_2013['1999'].iloc[0] - df_1998_a_2013['1998'].iloc[0] > 0 else 0
            valor_1999_2000 = 1 if df_1998_a_2013['2000'].iloc[0] - df_1998_a_2013['1999'].iloc[0] > 0 else 0
            valor_2000_2001 = 1 if df_1998_a_2013['2001'].iloc[0] - df_1998_a_2013['2000'].iloc[0] > 0 else 0
            valor_2001_2002 = 1 if df_1998_a_2013['2002'].iloc[0] - df_1998_a_2013['2001'].iloc[0] > 0 else 0
            valor_2002_2003 = 1 if df_1998_a_2013['2003'].iloc[0] - df_1998_a_2013['2002'].iloc[0] > 0 else 0
            valor_2003_2004 = 1 if df_1998_a_2013['2004'].iloc[0] - df_1998_a_2013['2003'].iloc[0] > 0 else 0
            valor_2004_2005 = 1 if df_1998_a_2013['2005'].iloc[0] - df_1998_a_2013['2004'].iloc[0] > 0 else 0
            valor_2005_2006 = 1 if df_1998_a_2013['2006'].iloc[0] - df_1998_a_2013['2005'].iloc[0] > 0 else 0
            valor_2006_2007 = 1 if df_1998_a_2013['2007'].iloc[0] - df_1998_a_2013['2006'].iloc[0] > 0 else 0
            valor_2007_2008 = 1 if df_1998_a_2013['2008'].iloc[0] - df_1998_a_2013['2007'].iloc[0] > 0 else 0
            valor_2008_2009 = 1 if df_1998_a_2013['2009'].iloc[0] - df_1998_a_2013['2008'].iloc[0] > 0 else 0
            valor_2009_2010 = 1 if df_1998_a_2013['2010'].iloc[0] - df_1998_a_2013['2009'].iloc[0] > 0 else 0
            valor_2010_2011 = 1 if df_1998_a_2013['2011'].iloc[0] - df_1998_a_2013['2010'].iloc[0] > 0 else 0
            valor_2011_2012 = 1 if df_1998_a_2013['2012'].iloc[0] - df_1998_a_2013['2011'].iloc[0] > 0 else 0
            valor_2012_2013 = 1 if df_1998_a_2013['2013'].iloc[0] - df_1998_a_2013['2012'].iloc[0] > 0 else 0
            
            valores = [pais, value, valor_1998_1999, valor_1999_2000, valor_2000_2001, valor_2001_2002, 
                       valor_2002_2003, valor_2003_2004, valor_2004_2005, valor_2005_2006, valor_2006_2007, 
                       valor_2007_2008, valor_2008_2009, valor_2009_2010, valor_2010_2011, valor_2011_2012, 
                       valor_2012_2013]

            df_auxiliar = pd.DataFrame([valores], columns=colunas_transformacao)
            df_auxiliar.index = range(df_auxiliar.shape[0])
            df_classificado = pd.concat([df_auxiliar, df_classificado])
        
df_classificado = df_classificado.sort_values(['Pais'], ascending=[True])
df_classificado.index = range(df_classificado.shape[0])
df_classificado

,Pais,Indicador,1998-1999,1999-2000,2000-2001,2001-2002,2002-2003,2003-2004,2004-2005,2005-2006,2006-2007,2007-2008,2008-2009,2009-2010,2010-2011,2011-2012,2012-2013
0,Argentina,UIS.XSPENDP.FDPUB.FNS,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
1,Argentina,UIS.LP.AG15T24,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
2,Argentina,UIS.ILLPOP.AG25T64,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
3,Argentina,SE.XPD.TOTL.GD.ZS,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
4,Argentina,UIS.XSPENDP.FDPUB.FNCAP,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,Uruguay,SE.XPD.TOTL.GB.ZS,0,1,0,1,1,1,1,0,0,0,0,0,1,0,0
347,Uruguay,UIS.XGDP.FSGOV.FDINSTADM.FFD,0,1,0,1,1,1,1,0,0,0,0,0,1,0,0
348,Uruguay,SE.XPD.TOTL.GD.ZS,0,1,0,1,1,1,1,0,0,0,0,0,1,0,0
349,Uruguay,UIS.ILLPOP.AG25T64,0,1,0,1,1,1,1,0,0,0,0,0,1,0,0


## 3.1.2.1) Validação da eficiência da previsão

In [340]:
df_classificado.describe().round(2)

,Pais,Indicador,1998-1999,1999-2000,2000-2001,2001-2002,2002-2003,2003-2004,2004-2005,2005-2006,2006-2007,2007-2008,2008-2009,2009-2010,2010-2011,2011-2012,2012-2013
count,351,351,351,351,351,351,351,351,351,351,351,351,351,351,351,351,351
unique,39,9,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,Switzerland,UIS.LP.AG15T24,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
freq,9,39,261,216,252,216,198,207,189,252,207,252,225,216,180,198,243
